In [14]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn import model_selection
tf.enable_eager_execution() #开启eager模式

In [ ]:
'''
高志阳-2016011471
任务说明：
将鸢尾花数据集安装8 : 2的比例划分成训练集与验证集（数据集部分可使用Dataset API，也可以不使用）。

设计模型：
使用TensorFlow设计K近邻模型（可不使用KD树优化算法）
模型关键部分需添加注释

训练模型：
使用TensorFlow完成训练相关的代码
训练关键部分需添加注释

验证模型：
使用验证集检测模型性能
使用验证集调整超参数
提交模型：

可使用Eager模式设计模型
'''

In [3]:
#加载鸢尾花数据集
iris = datasets.load_iris()
#将数据集用留出法划分为两部分
# 指定20%的数据集作为测试集，划分前打乱数据
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    iris.data, iris.target, test_size=0.2, shuffle=True)

In [4]:
#设计K近邻模型(TensorFlow)（K=5）
#求两点之间距离(二三维)
def x_y(x1, x2):
    x_dis = tf.square(tf.subtract(x1[2], x2[2]))
    y_dis = tf.square(tf.subtract(x1[3], x2[3]))
    x_y_dis = tf.add(x_dis, y_dis)
    dis = tf.sqrt(x_y_dis)
    return dis

def panduan_x0(x0_test, k):
    distance = []  #保存距离
    y_distance = []  #保存标签
    
    #先保存前k个点的距离
    for q_i in tf.range(k):
        dis = x_y(x0_test, x_train[q_i])
        distance.append(dis)
        y_distance.append(y_train[q_i])

    #排序(小->大)
    for ci in tf.range(k-1):
        for cj in tf.range(k-1):
            if tf.greater(distance[cj],distance[cj+1]):
                temp = distance[cj]
                distance[cj] = distance[cj+1]
                distance[cj+1] = temp
                y_temp = y_distance[cj]
                y_distance[cj] = y_distance[cj+1]
                y_distance[cj+1] = y_temp

    #分别计算x0_test与每个数据之间的欧式距离
    for i in tf.range(120-k):
        dis = x_y(x0_test, x_train[i+k])
        
        #判断当前距离是不是最近的并保存在distance中
        dis_i = 0
        for ik in tf.range(k):
            if tf.less(dis_i,k):
                if tf.less(dis,distance[dis_i]):
                    distance[dis_i] = dis
                    y_distance[dis_i] = y_train[i+k]
                    dis_i = k
                else:
                    dis_i = dis_i+1
            
        #判断当前所属类别
        num_0 = tf.Variable(tf.constant(0))
        num_1 = tf.Variable(tf.constant(0))
        num_2 = tf.Variable(tf.constant(0))
        for i in tf.range(k):
            if y_distance[i]==0:
                num_0 = num_0+1
            elif y_distance[i]<2:
                num_1 = num_1+1
            else:
                num_2 = num_2+1
        y_true = tf.Variable(tf.constant(0))
        if num_0>num_1:
            if num_0>num_2:
                y_true = 0
            else:
                y_true = 2
        else:
            if num_1>num_2:
                y_true = 1
            else:
                y_true = 2
    #返回所属的类别
    return y_true

In [49]:
#得到不同k对应的y_list
def ru_y_list():
    y_list = []
    for k in tf.range(1, 10):
        #调用算法测试x_test
        y_linshi = []
        for m in x_test:
            y_linshi.append(panduan_x0(m, k))
        y_list.append(y_linshi)
    return y_list
#得到最高的正确率对应的k
def ru_ri(y_list):
    #计算正确率
    rn_list = []
    r_k = 0
    r_y = []
    for n in tf.range(9):
        r_n = 0
        for m in tf.range(30):
            if y_test[m] == y_list[n][m]:
                r_n = r_n+1
        rn_list.append(r_n)
    for rn in tf.range(8):
        if rn_list[rn] > rn_list[rn+1]:
            r_k = rn
            r_y = y_list[rn]
    return r_k, r_y
#计算正确率
def zql(y):
    num = 0
    for z in tf.range(30):
        if y[z] == y_test[z]:
            num = num+1
    return num/30

In [42]:
y_list = ru_y_list()
y_list = np.array(y_list)

In [53]:
r_k, y = ru_ri(y_list)
print(r_k)
print(r_y)

tf.Tensor(1, shape=(), dtype=int32)
[2 1 2 1 0 1 2 1 1 2 1 0 2 2 0 0 0 2 2 1 2 1 0 1 2 0 2 0 1 1]


In [54]:
rig = zql(y)

In [56]:
print('K的最优值为',r_k)
print("k近邻算法得y_test为：",y)
print("y_test为",y_test)
print('正确率为',rig)

K的最优值为 tf.Tensor(1, shape=(), dtype=int32)
k近邻算法得y_test为： [2 1 2 1 0 1 2 1 1 2 1 0 2 2 0 0 0 2 2 1 2 1 0 1 2 0 2 0 1 1]
y_test为 [2 1 2 1 0 1 2 1 1 2 1 0 2 2 0 0 0 2 2 1 2 1 0 1 2 0 2 0 1 2]
正确率为 0.9666666666666667
